In [ ]:
## Python libs

import math
import torch
import pickle
import warnings
import numpy as np
import random

import matplotlib.pyplot as plt
%matplotlib inline

from pathlib import Path
from sklearn.datasets import load_svmlight_file

warnings.simplefilter('ignore')

In [ ]:
COLAB = False

## There must be data 'vladiku.zip' and zip of repository
## https://github.com/alexrogozin12/decentralized_methods (private at the date of coding)
## in google-drive folder 'GDDIR'
GDDIR = 'uploads/rogozin'

if COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')

    !cp -r /content/gdrive/My\ Drive/{GDDIR} .
    !unzip -qn {Path(GDDIR).name}/vladiku.zip -d data
    !bunzip2 data/*.bz2

    !git clone https://github.com/alexrogozin12/decentralized_methods.git
    !mv decentralized_methods/* .
    !rm decentralized_methods -rf
    !sed -ri '8d' src/utils.py

# DGM Minimal Environment 

In [ ]:
## Local libs

from src.objectives import ( 
    LeastSquares, LogRegression,
    StochLeastSquares, StochLogRegression)
from src.methods import (
    EXTRA, DIGing, DSGD,
    DAccGD, Mudag, APM_C,
    SDAccGD, SMudag, SAPM_C)
from src.utils import PythonGraph, expected_lambda2

In [ ]:
TASK = LeastSquares
# TASK = LogRegression 

DDIR = 'logreg_solutions' if TASK == LogRegression else 'least_squares_solutions' 
DSDIR = Path('data/cadata_scaled')
DDIR = Path(DDIR)

num_nodes = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
soldir = DDIR / DSDIR.name

A, b = load_svmlight_file(str(DSDIR))

A_cpu = torch.Tensor(A.todense())
b_cpu = torch.Tensor(b)

A = A_cpu.to(device)
b = b_cpu.to(device)

In [ ]:
fname = list(soldir.iterdir())[0].name
sigma = float(fname.split('=')[1])

with open(soldir/fname, 'rb') as file:
    f_star = pickle.load(file)['func_star']

In [ ]:
# For simulating a graph evolution,
# only graphs like 'erdos_renyi' and 'random_geometric' are appropriate

p = .68
graph = 'random_geometric'
# graph = 'erdos_renyi'
# graph = 'path'
# graph = 'cycle'
# graph = 'complete'

F = TASK(A, b, num_nodes, sigma)
F_cpu = TASK(A_cpu, b_cpu, num_nodes, sigma)
X0 = torch.zeros(num_nodes, A.size(1)).to(device)

In [ ]:
L = torch.svd(A)[1][0] ** 2 / (4*len(A))
kappa_g = torch.svd(F.A)[1][:, 0].mean() / sigma

In [ ]:
gen = lambda : PythonGraph(F, graph, p).gen()[1]
_gen = lambda: PythonGraph(F_cpu, graph, p).gen()[1]
E_s2,_ = expected_lambda2(_gen, 5000)

####
# Fixing seed doesn't really make a difference
####
# torch.manual_seed(123)  #  I don't remember whether I use torch random numbers anywhere
# random.seed(123)  #  networkx depends on lib random
# graphs = [PythonGraph(F, graph, p).gen()[1] for _ in range(int(1e4))]

# class GraphEvolution:
#     def __init__(self, graphs):
#         self.gi = iter(graphs)
        
#     def __call__(self):
#         return next(self.gi)

In [ ]:
opts = []
opts.append(
    EXTRA(F, gen, eta=.01/F.b.norm()))

opts.append(
    DIGing(F, gen, eta=.01/F.b.norm()))

consensus_iters = 5
opts.append(
    DAccGD(F, gen, L=L, mu=sigma, con_iters=consensus_iters))

# consensus_iters = 5
# opts.append(
#     SDAccGD(F, gen, L=L, mu=sigma, E_s2=E_s2, con_iters=consensus_iters))

consensus_iters = 20
opts.append(
    SMudag(F, gen, L=L, mu=sigma, E_s2=E_s2, con_iters=consensus_iters))

opts.append(
    SAPM_C(F, gen, L=L, mu=sigma, E_s2=E_s2, beta=0, scale=20000))  # NOTE: tune up 'scale'

checkpoints = [[X0] for _ in range(len(opts))]

In [ ]:
# Pay attention to the values of (EXTRA/DIGing).eta, SAPM_C._gamma (gamma depends on 'scale')
print('EXTRA/DIGing eta', opts[0].eta.item())
print('APM-C gamma', opts[-1]._gamma.item())
assert opts[-1]._gamma <= 1, 'Too long to wait'

In [ ]:
## Fixed mixing matrix algorithms
## EXTRA and DIGing with a small correction in the code can be also regarded as such

# opts = []
# W = gen()
# opts.append(
#     Mudag(F, W, L=L, mu=sigma, M=L, kappa_g=kappa_g, scale=1.))  # NOTE: tune up 'scale'
# 
# beta = 0
# opts.append(
#     APM_C(F, W, L=L, mu=sigma, beta=beta, scale=100))  # NOTE: tune up 'scale'

In [ ]:
%%time
## Running the cell X times yields X * n_iters optimization steps of each optimizer
## (if n_iters is not redefined during it). To run from scratch,
## execute the cell with optimizers' initialization first

n_iters = 500

for i, opt in enumerate(opts):
    X0, *args = checkpoints[i]
    checkpoints[i] = opt.run(X0, *args, n_iters=n_iters);

In [ ]:
def name_corrector(names):
    corrected_names = []
    for name in names:
        if name[0] == 'S': new_name = name[1:]
        else: new_name = name
            
        if new_name in names: new_name = name 
        new_name = new_name.replace('_', '-')
        
        corrected_names.append(new_name)
    return corrected_names

In [ ]:
XAXIS = 'nmix'
# XAXIS = 'i'
XLIM = n_iters if XAXIS == 'i' else min([opt.logs['nmix'][-1] for opt in opts])
XLBL = 'gradient steps' if XAXIS == 'i' else 'communication steps'

plt.figure(figsize=(8, 8))
lbls = [opt.__class__.__name__ for opt in opts]
lbls = name_corrector(lbls)

for i, opt in enumerate(opts):
    span = np.searchsorted(opt.logs[XAXIS], XLIM, 'right')
    plt.plot(
        opt.logs[XAXIS][:span], opt.logs['fn'][:span] - f_star,
        marker=i+5, label=lbls[i], markevery=span//(4+math.ceil(i*1.8))+1)
    
plt.ylabel(r'$f(\overline{x}_k) - f^*$', size=20)
plt.xlabel(XLBL, size=20)

#plt.ylim(0, 1.2*opts[0].logs['fn'][0])
plt.yscale('log')
plt.grid()
plt.legend();

if COLAB:
    plt.savefig(f'{DSDIR.name}-res_x{XAXIS}.png')
    !mv {DSDIR.name}-res_x{XAXIS}.png /content/gdrive/My\ Drive/{GDDIR}/figures
else:
    !mkdir -p figures
    plt.savefig(f'figures/{DSDIR.name}-res_x{XAXIS}.png')

In [ ]:
XAXIS = 'nmix'
# XAXIS = 'i'
XLIM = n_iters if XAXIS == 'i' else min([opt.logs['nmix'][-1] for opt in opts])
XLBL = 'gradient steps' if XAXIS == 'i' else 'communication steps'

plt.figure(figsize=(8, 8))
lbls = [opt.__class__.__name__ for opt in opts]
lbls = name_corrector(lbls)

for i, opt in enumerate(opts):
    span = np.searchsorted(opt.logs[XAXIS], XLIM, 'right')
    plt.plot(
        opt.logs[XAXIS][:span], opt.logs['dist2con'][:span],
        marker=i+5, label=lbls[i], markevery=span//(3+math.ceil(i*1.7))+1)

plt.ylabel(r'$||(I-\frac{1}{n}11^T)X||^2$', size=20)
plt.xlabel(XLBL, size=20)

plt.yscale('log')
plt.legend()
plt.grid();

if COLAB:
    plt.savefig(f'{DSDIR.name}-con_x{XAXIS}.png')
    !mv {DSDIR.name}-con_x{XAXIS}.png /content/gdrive/My\ Drive/{GDDIR}/figures
else:
    !mkdir -p figures
    plt.savefig(f'figures/{DSDIR.name}-con_x{XAXIS}.png')